In [ ]:
%matplotlib inline
from vis_common import *

In [ ]:
f[ilost].groupby('level').size().plot.bar(title='Deaths by player level');

In [ ]:
# TODO: bleh, lots of junk places in here. Should probably check against a fixed set when parsing.
x = f[ilost].groupby(['level', 'wheredied']).size().unstack()
x = x.div(x.sum(axis=1), axis=0)
# That legend is crazy, and probably need to normalize, but this isn't half-bad
x.plot.area(figsize=(13,9), stacked=True, legend=1);

In [ ]:
visit_cols = [col for col in f.columns if col.startswith('visited_')]
print visit_cols
f.groupby('wheredied').size().where(lambda x: x > 0).dropna().head()

In [ ]:
visited_to_wheredied = {
    'bailey': 'bailey',
    'desolation': 'desolation of salt',
    'ice cave': 'ice cave',
    'labyrinth': 'labyrinth',
    'ossuary': 'ossuary',
    'sewer': 'sewer',
    'volcano': 'volcano',
    'wizlab': "wizard's laboratory",
}

# Probably a clever way to do this w/o explicit looping, but whatever
vault_deadliness = {}
for (v, w) in visited_to_wheredied.iteritems():
    deaths = len(f[f['wheredied'] == w])
    visits = len(f[f['visited_'+v] == True])
    vault_deadliness[v] = 100 * deaths/(visits+0.0)
print vault_deadliness

In [ ]:
plot_dict(vault_deadliness, 'Portal vault deadliness (% of entrants killed)');
# Main trend here seems to be that vaults that appear later in the game have lower
# lethality. This is probably less a property of the vaults themselves and more a
# byproduct of how deaths are front-loaded to early player levels, and that players who
# are able to make it as far as a wizlab are very skilled.
# Would be cool to have some notion of 'level-adjusted deadliness'. Also for monsters.

In [ ]:
n_players = len(f)
n_deathplace = f['wheredied'].count()
n_losers = len(f[ilost])
print "{} / {} losers' place of death unaccounted for".format(n_losers-n_deathplace, n_losers)
# Hm, that's like 10%? That's very worrying. Can those really all be players who quit/escaped?
f[ilost & f['wheredied'].isnull()].groupby('level').size()
# Maybe? Worth coming back to this once cause of death is recorded.

In [ ]:
f.groupby('howdied').size().sort_values(ascending=False)

One possible takeaway from the above is that starvation isn't really much of an existential threat in crawl, and that maybe food should be made more rare. While I do think this is true, the above doesn't really represent a strong argument for that, because you might be more likely to die to a monster as a result of the symptoms of being on the brink of starvation death (nocast, combat penalties, fainting) before you die of starvation directly.

Could test that case by recording satiety level on death and counting how often people die while at 'starving' level hunger.

In [ ]:
f[f['saw_temple']==True].groupby('howdied').size().sort_values(ascending=False)

In [ ]:
f[iquit].groupby('level').size().plot.bar();
# A lot of people quit at level 1, and I don't know why. Like a loooot.